In [ ]:
from scipy.io import loadmat
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
import torch, gc
import skimage
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
import pandas as pd
from PIL import Image
import torch
import torchvision
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.autograd import Variable
import tqdm
from tqdm import tqdm_notebook
import scipy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls drive/MyDrive/University/Science

In [ ]:
%cd drive/MyDrive/University/Science/code/NucleiSegmentationAI

In [ ]:
!ls

In [ ]:
############################################
# Display Image and its corresponding Mask #
############################################
x = loadmat('./Train/Labels/train_1.mat')
img=mpimg.imread('./Train/Images/train_1.png')
imgplot = plt.imshow(img)
plt.show()
imshow(np.pad((x['inst_map']>=1).astype(int),12), cmap='Purples')

In [ ]:
#grey
class NucleiDataset(Dataset):
    def __init__(self, image_path, mask_path, crop_size = 256, k=700, transform=None, draw=False, ls=64, us=8):
        self.images = []
        self.masks = []
        self.image_types = []
        images = []
        masks = []
        img_ls = [image_path+i for i in sorted(os.listdir(image_path))]

        mask_ls = [mask_path+i for i in sorted(os.listdir(mask_path))]
        self.transform = transform
        for img_name, mask_name in zip (img_ls, mask_ls):
          img = Image.open(img_name).convert('RGB')
          img.load()
          mask = np.pad((loadmat(mask_name)['inst_map']>=1).astype(int),12)
          #7 types
          if self.transform:
              img = self.transform(img)
          images.append(img)
          masks.append(mask)
        l = len(images)

        for _ in range(ls):
          k1 = random.randint(0, l-1)
          image = images[k1]
          mask = masks[k1]
          _, h, w = image.shape
          x = np.random.randint(0, w - crop_size -1)
          y = np.random.randint(k, h - crop_size -1)
          if draw:
              draw()
          # Обрезка изображения
          self.images.append(image[:, y:y+crop_size, x:x+crop_size])

          # Обрезка маски
          self.masks.append(mask[y:y+crop_size, x:x+crop_size])
          self.image_types.append('labeled')
          #print(img.shape, mask.shape)
        for _ in range(us):
          k1 = random.randint(0, l-1)
          image = images[k1]
          x = np.random.randint(0, w - crop_size - 1)
          y = np.random.randint(0, k - crop_size - 1)
          # Обрезка изображения
          self.images.append(image[:, y:y+crop_size, x:x+crop_size])
          self.masks.append(mask[y:y+crop_size, x:x+crop_size])
          self.image_types.append('unlabeled')


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        mask = self.masks[idx]
        image_type = self.image_types[idx]
        return img, mask, image_type#mask[np.newaxis, ...]


trans = transforms.Compose([
    transforms.Pad(12),    # given image is 1000x1000, pad it to make it 1024x1024
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # imagenet normalization
])

train_trans = transforms.Compose([
    transforms.Pad(12),    # given image is 1000x1000, pad it to make it 1024x1024
    transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.RandomPerspective(distortion_scale=0.6, p=1.0),
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # imagenet normalization
])

label_path = "./Train/Labels/"
img_path = "./Train/Images/"

batch_size = 72  #my gpu is 8gb, using batchsize of 2 already insufficient memory, so i use batch size 1
labeled_size = 8
unlabaled_size = batch_size - labeled_size
train_set = NucleiDataset(img_path, label_path, transform=trans, ls=labeled_size, us=unlabaled_size)



train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False, num_workers=1, drop_last=True)

In [ ]:
def reverse_transform(inp, mask):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    inp = (inp * 255).astype(np.uint8)
    print(inp.shape, mask.shape)
    inp[mask > 0] = (0, 255, 255)

    return inp


inputs, masks, image_type = next(iter(train_loader))

print(inputs.shape, masks.shape)  #shapes of our inputs to the model and loss function

plt.imshow(reverse_transform(inputs[0], masks[0]))
plt.show()
imshow(masks[0],cmap='gray')

In [ ]:
######################
# Model Architecture #
######################

def convrelu(in_channels, out_channels, kernel, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
        nn.ReLU(inplace=True),
    )


class ResNetUNet(nn.Module):
    def __init__(self, n_class):
        super().__init__()

        self.base_model = models.resnet18(pretrained=True)
        self.base_layers = list(self.base_model.children())

        self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
        self.layer0_1x1 = convrelu(64, 64, 1, 0)
        self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)
        self.layer1_1x1 = convrelu(64, 64, 1, 0)
        self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)
        self.layer2_1x1 = convrelu(128, 128, 1, 0)
        self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)
        self.layer3_1x1 = convrelu(256, 256, 1, 0)
        self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
        self.layer4_1x1 = convrelu(512, 512, 1, 0)

        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
        self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
        self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
        self.conv_up0 = convrelu(64 + 256, 128, 3, 1)

        self.conv_original_size0 = convrelu(3, 64, 3, 1)
        self.conv_original_size1 = convrelu(64, 64, 3, 1)
        self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)

        self.conv_last = nn.Conv2d(64, n_class, 1)

    def forward(self, input):
        x_original = self.conv_original_size0(input)
        x_original = self.conv_original_size1(x_original)

        layer0 = self.layer0(input)
        layer1 = self.layer1(layer0)
        layer2 = self.layer2(layer1)
        layer3 = self.layer3(layer2)
        layer4 = self.layer4(layer3)

        layer4 = self.layer4_1x1(layer4)
        x = self.upsample(layer4)

        layer3 = self.layer3_1x1(layer3)
        x = torch.cat([x, layer3], dim=1)
        x = self.conv_up3(x)

        x = self.upsample(x)
        layer2 = self.layer2_1x1(layer2)
        x = torch.cat([x, layer2], dim=1)
        x = self.conv_up2(x)

        x = self.upsample(x)
        layer1 = self.layer1_1x1(layer1)
        x = torch.cat([x, layer1], dim=1)
        x = self.conv_up1(x)

        x = self.upsample(x)
        layer0 = self.layer0_1x1(layer0)
        x = torch.cat([x, layer0], dim=1)
        x = self.conv_up0(x)

        x = self.upsample(x)
        x = torch.cat([x, x_original], dim=1)
        x = self.conv_original_size2(x)

        out = self.conv_last(x)

        return out

In [ ]:
###################################################################
# Loss function that combines Binary Cross Entropy with Dice loss #
###################################################################

# Model output is (N, 1, 1024,1024), where N = batch size, Thus you will see that i squeeze the 2nd dimension

def weighted_loss(pred, targ, bce_weight=0.5, smooth=1., binary=True):

    if binary:
      bce = F.binary_cross_entropy_with_logits(pred.squeeze(dim=1), targ)
    else:
      bce = F.cross_entropy(pred, targ)
    pred = torch.sigmoid(pred)

    pred = pred.contiguous().squeeze(dim=1)
    targ = targ.contiguous()

    intersection = (pred * targ).sum(dim=1).sum(dim=1)
    dice = (1 - ((2. * intersection + smooth) / (pred.sum(dim=1).sum(dim=1) + targ.sum(dim=1).sum(dim=1) + smooth)))
    loss = bce * bce_weight + dice.mean() * (1 - bce_weight)
    #intersection = (pred * targ).sum(dim=1).sum(dim=1)
    #dice = (1 - ((2. * intersection + smooth) / (pred.sum(dim=1).sum(dim=1) + targ.sum(dim=1).sum(dim=1) + smooth)))
    return loss, dice, bce

In [ ]:
def update_ema_variables(alpha, global_step):
    # Use the true average until the exponential average is more correct
    alpha = min(1 - 1 / (global_step + 1), alpha)
    for ema_param, param in zip(ema_model.parameters(), model.parameters()):
        ema_param.data.mul_(alpha).add_(1 - alpha, param.data)



In [ ]:
def sigmoid_rampup(current, rampup_length):
  if rampup_length == 0:
      return 1.0
  else:
      current = np.clip(current, 0.0, rampup_length)
      phase = 1.0 - current / rampup_length
      return float(np.exp(-5.0 * phase * phase))

def get_current_consistency_weight(epoch):
    # Consistency ramp-up from https://arxiv.org/abs/1610.02242
    consistency = 0.1
    consistency_rampup = 200.0
    return consistency * sigmoid_rampup(epoch, consistency_rampup)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import gc

In [ ]:
def metrics_curve(loss_ls, dice_ls, bce_ls):



        plt.plot(loss_ls)

        print("loss curve")
        fig.savefig("curr_loss.png", dpi=fig.dpi)
        plt.show()
        plt.clf()

        print("dice loss curve")
        plt.plot(dice_ls)
        plt.show()
        plt.clf()

        print("cross entropy loss")
        plt.plot(bce_ls)
        plt.show()
        plt.clf()

def log_losses(supervised_loss, dice, bce, consistency_loss, loss):
        print("epoch supervised loss:", supervised_loss)
        print("epoch dice:", dice.mean())
        print("epoch bce:", bce)
        print("unsupervised loss", consistency_loss)
        print("loss", loss)


def draw_image_and_mask(inputs, outputs, masks):
        print("\n","Input Image", inputs.to('cpu').detach()[0].shape, outputs.to('cpu').detach()[0].shape)
        plt.imshow(reverse_transform(inputs.to('cpu').detach()[0], masks.to('cpu').detach()[0]))
        plt.show()
        print("Predicted Mask Sigmoid")
        plt.imshow(outputs.to('cpu').detach().numpy()[0][0])
        plt.show()
        print("True mask")
        plt.imshow(masks.to('cpu').detach()[0]) #, cmap='gray'
        plt.show()

def draw():

    overlay = np.array(image.cpu().detach().numpy(), dtype = np.uint8).transpose(1, 2, 0)

    np_image = overlay
    overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



    cv2.rectangle(overlay, (0, 0), (w, h-k), (0, 200, 0), -1)
    alpha = 0.3  # Transparency factor.

    # Following line overlays transparent rectangle
    # over the image
    image_new = cv2.addWeighted(overlay, alpha, np_image, 1 - alpha, 0)
    cv2_imshow(image_new)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    overlay = image_new.copy()
    cv2.rectangle(overlay, (0, h-k), (w, h), (100, 0, 50), -1)

    alpha = 0.3  # Transparency factor.
    # Following line overlays transparent rectangle
    # over the image
    image_new = cv2.addWeighted(overlay, alpha, image_new, 1 - alpha, 0)
    cv2_imshow(image_new)
    cv2.waitKey(0)

In [ ]:
def mix_unlabeled(unlabeled_bs, unlabeled_volume_batch_0, unlabeled_volume_batch_1):
      ict_alpha = 0.21
      ict_mix_factors = np.random.beta(
          ict_alpha, ict_alpha, size=((unlabeled_bs // 2, 1, 1, 1)))
      ict_mix_factors = torch.tensor(
          ict_mix_factors, dtype=torch.float).cuda()

      batch_ux_mixed = unlabeled_volume_batch_0 * \
          (1.0 - ict_mix_factors) + \
          unlabeled_volume_batch_1 * ict_mix_factors

      return batch_ux_mixed, ict_mix_factors

In [ ]:
fig = plt.figure()
import random
import torchvision.transforms as T
from google.colab.patches import cv2_imshow
pil_transform = T.ToPILImage()
def train_model(optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10

    ema_decay = 0.99
    labeled_bs = 8
    loss_ls = []
    dice_ls = []
    bce_ls = []
    iter_num = 0
    unlabeled_bs = 64
    crop_size = 256
    draw = False

    k = 700


    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        since = time.time()
        supervised_loss = 0


        for inputs, masks, image_type in train_loader:
            print(inputs.shape, masks.shape, inputs[0].shape, masks[0].shape)
            labeled_images = []
            labeled_masks = []
            unlabeled_images = []
            model.train()

            labeled_images = inputs[:8].to(device)
            unlabeled_images = inputs[8:].to(device)
            labeled_masks = masks[:8].to(device).float()

            #ICT mix factors
            print("shapws", labeled_images.shape, unlabeled_images.shape)

            unlabeled_volume_batch_0 = unlabeled_images[0: unlabeled_bs // 2, ...]
            unlabeled_volume_batch_1 = unlabeled_images[unlabeled_bs // 2:, ...]


            #Mix images
            batch_ux_mixed, ict_mix_factors = mix_unlabeled(unlabeled_bs, unlabeled_volume_batch_0, unlabeled_volume_batch_1)

            input_volume_batch = torch.cat(
                [labeled_images, batch_ux_mixed], dim=0)
            outputs = model(input_volume_batch)

            with torch.no_grad():
              ema_output_ux0 = ema_model(unlabeled_volume_batch_0)
              ema_output_ux1 = ema_model(unlabeled_volume_batch_1)
              batch_pred_mixed = ema_output_ux0 * \
                  (1.0 - ict_mix_factors) + ema_output_ux1 * ict_mix_factors
            #print(outputs[:labeled_bs].shape, labeled_masks[:labeled_bs].shape)
            supervised_loss, dice, bce = weighted_loss(outputs[:labeled_bs], labeled_masks[:labeled_bs], bce_weight=0.5)
            iter_num += 1

            consistency_weight = get_current_consistency_weight(iter_num // 150)
            consistency_loss = torch.mean(
                (outputs[labeled_bs:] - batch_pred_mixed) ** 2)
            loss = supervised_loss + consistency_weight * consistency_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            update_ema_variables(ema_decay, iter_num)

            dice_ls.append(dice.mean().item())
            loss_ls.append(loss.item())
            bce_ls.append(bce.item())

            log_losses(supervised_loss, dice, bce, consistency_loss, loss)

            metrics_curve(loss_ls, dice_ls, bce_ls)


        if (epoch+1)%4==0:
            draw_image_and_mask(inputs, outputs, masks)

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60),"\n")


    return loss_ls, model

In [ ]:
def create_model(ema=False):
  # Network definition
  model = ResNetUNet(1).to(device)
  if ema:
    for param in model.parameters():
        param.detach_()
  return model


model = create_model()
#model.load_state_dict(torch.load('semi-supervised.pt', map_location=torch.device('cpu')))
ema_model = create_model(ema=True)


optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1)

loss_ls = train_model(optimizer_ft, exp_lr_scheduler, num_epochs=500)

In [ ]:
def create_model(ema=False):
  # Network definition
  model = ResNetUNet(1).to(device)
  if ema:
    for param in model.parameters():
        param.detach_()
  return model


model = create_model()

ema_model = create_model(ema=True)


optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1)

loss_ls = train_model(optimizer_ft, exp_lr_scheduler, num_epochs=60)

In [ ]:
torch.save(model.state_dict(), 'semi-supervised-1classes.pt')

In [ ]:
del train_loader

In [ ]:
#######################
# Test Set Dataloader #
#######################

label_path = "./Test/Labels/"
img_path = "./Test/Images/"

test_set = NucleiDataset(img_path,label_path, transform = trans)

batch_size = 1

test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=1)

In [ ]:
model.eval()

In [ ]:
#inputs, masks = next(iter(test_loader))
#from torchvision.utils import draw_segmentation_masks
dices = []
losses = []
smooth=1.
crop_size = 256
for inputs, masks, t in test_loader:
  print("Input Image")

  plt.show()
  outputs = model(inputs.to(device))

  outp = torch.sigmoid(outputs).to('cpu')


  outp = outp.contiguous().squeeze(dim=1)

  targ = masks.contiguous().to('cpu')

  intersection = (outp * targ).sum(dim=1).sum(dim=1)

  dice = (1 - ((2. * intersection + smooth) / (outp.sum(dim=1).sum(dim=1) + targ.sum(dim=1).sum(dim=1) + smooth)))

  print("dice", dice.mean())


  pred = outputs.to('cpu').detach().numpy()[0][0]

  #draw_segmentation_masks(inputs, masks)
  #draw_segmentation_masks(inputs, masks)

  #preds = torch.argmax(softmax(outputs),axis=1).to('cpu')


  print("Predicted Mask Sigmoid")
  plt.imshow(pred)

  plt.show()
  threshold=0.95     # vary the threshold
  pred[pred >= threshold] = 1
  pred[pred < threshold] = 0

  #pred[pred_1 < threshold  pred_2 < threshold  pred_3 < threshold] = 0

  print("Predicted Mask Binary")
  plt.imshow(pred.T)
  plt.show()
  print("Actual Mask")
  plt.imshow(masks[0].T)
  plt.show()
  dices.append(dice.mean().item())
  print(dices)
  #dices.append(dice)
  #losses.append(loss)pred

  del outputs
  del outp
  del targ
  del pred

  gc.collect()
  torch.cuda.empty_cache()
print("final dice", sum(dices) / len(dices))

In [ ]:
dice_arr = [0.2202581763267517, 0.3833320736885071, 0.23471367359161377, 0.28936779499053955, 0.2180195450782776, 0.267808735370636, 0.21645820140838623, 0.21842467784881592, 0.23607689142227173, 0.23001813888549805, 0.27057749032974243, 0.24977004528045654, 0.25484317541122437, 0.18952924013137817]

In [ ]:
sum(dice_arr) / len(dice_arr)